In [1]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt



In [15]:
# Load Transiting Spectrum:
transit_file = "EPIC 211822797_med-58507-TD084806N172341K01_sp14-051.fits"
data_transit = fits.open(transit_file)

In [16]:
#checking the data structure
data_transit.info()

Filename: EPIC 211822797_med-58507-TD084806N172341K01_sp14-051.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  Information    1 PrimaryHDU      91   ()      
  1  COADD_B       1 BinTableHDU     38   1R x 6C   [4104E, 4104E, 4104E, 4104I, 4104I, 4104D]   
  2  COADD_R       1 BinTableHDU     38   1R x 6C   [3854E, 3854E, 3854E, 3854I, 3854I, 3854D]   
  3  B-84250145    1 BinTableHDU     36   1R x 5C   [4136E, 4136E, 4136E, 4136I, 4136D]   
  4  B-84250168    1 BinTableHDU     36   1R x 5C   [4136E, 4136E, 4136E, 4136I, 4136D]   
  5  B-84250192    1 BinTableHDU     36   1R x 5C   [4136E, 4136E, 4136E, 4136I, 4136D]   
  6  B-84250215    1 BinTableHDU     36   1R x 5C   [4136E, 4136E, 4136E, 4136I, 4136D]   
  7  R-84250145    1 BinTableHDU     36   1R x 5C   [4136E, 4136E, 4136E, 4136I, 4136D]   
  8  R-84250168    1 BinTableHDU     36   1R x 5C   [4136E, 4136E, 4136E, 4136I, 4136D]   
  9  R-84250192    1 BinTableHDU     36   1R x 5C   [4136E, 4136E, 4136E, 41

In [23]:
# Red arm spectra
transit_wvr = data_transit[2].data["WAVELENGTH"]
transit_red = data_transit[2].data["FLUX"]

# Blue arm spectra
transit_wvb = data_transit[1].data["WAVELENGTH"]
transit_blue = data_transit[1].data["FLUX"]

# Calculate median flux for each arm
median_flux_red = np.median(transit_red)
median_flux_blue = np.median(transit_blue)

# Normalize spectra
normalized_red = transit_red / median_flux_red
normalized_blue = transit_blue / median_flux_blue

# Load Non-Transit Spectra:
non_transit_files = ["EPIC 211822797_med-58508-TD084806N172341K01_sp14-051.fits"]

non_transit_red_spectra = []
non_transit_blue_spectra = []

for file in non_transit_files:
    data_non_transit = fits.open(file)
    non_transit_red_spectra.append(data_non_transit[2].data["FLUX"])
    non_transit_blue_spectra.append(data_non_transit[1].data["FLUX"])

# Find the minimum length among all spectra
min_length_red = min(min(len(spec) for spec in non_transit_red_spectra), len(transit_red))
min_length_blue = min(min(len(spec) for spec in non_transit_blue_spectra), len(transit_blue))

def interpolate_spectrum(wavelength, flux, target_wavelength):
    # Remove NaN values
    valid_indices = ~np.isnan(flux)
    wavelength = wavelength[valid_indices]
    flux = flux[valid_indices]
    
    # Interpolate onto the target wavelength grid
    interpolated_flux = np.interp(target_wavelength, wavelength, flux, left=np.nan, right=np.nan)
    
    return interpolated_flux

# Trim or pad all spectra to the minimum length
non_transit_red_spectra = np.array([interpolate_spectrum(transit_wvr, spec[:min_length_red], transit_wvr) for spec in non_transit_red_spectra])
non_transit_blue_spectra = np.array([interpolate_spectrum(transit_wvb, spec[:min_length_blue], transit_wvb) for spec in non_transit_blue_spectra])


# Calculate mean spectra
mean_non_transit_red = np.nanmean(non_transit_red_spectra, axis=0)
mean_non_transit_blue = np.nanmean(non_transit_blue_spectra, axis=0)

# Divide In-Transit Spectrum by Mean Spectrum:
divided_transit_red = normalized_red / mean_non_transit_red
divided_transit_blue = normalized_blue / mean_non_transit_blue

# Binning for Increased Signal-to-Noise Ratio (SNR):
def bin_spectrum(wavelength, flux, bin_size):
    binned_wavelength = wavelength[:len(wavelength)//bin_size * bin_size].reshape(-1, bin_size).mean(axis=1)
    binned_flux = flux[:len(flux)//bin_size * bin_size].reshape(-1, bin_size).mean(axis=1)
    return binned_wavelength, binned_flux

bin_size = 10

binned_wvb, binned_normalized_blue = bin_spectrum(transit_wvb, divided_transit_blue, bin_size)
binned_wvr, binned_normalized_red = bin_spectrum(transit_wvr, divided_transit_red, bin_size)

# Plotting:
# Assuming you have defined other variables such as wvb, wvr, normalized_blue, normalized_red, etc.

# Find the range of normalized flux values
min_flux = min(np.min(normalized_blue), np.min(normalized_red))
max_flux = max(np.max(normalized_blue), np.max(normalized_red))

# Increase the figure width for stretching
f, (ax, ax2) = plt.subplots(1, 2, sharey=True, facecolor='w', figsize=(14, 6))
f.subplots_adjust(wspace=0.020)

# Plot the normalized spectra on both axes with a gap in the broken axis
ax.plot(binned_wvb[0,:], binned_normalized_blue[0,:], label='Normalized Blue Arm', color='blue')
ax2.plot(binned_wvr[0,:], binned_normalized_red[0,:], label='Normalized Red Arm', color='red')

# Add a gap in the broken axis using np.nan
ax.plot([np.nan, np.nan], [np.nan, np.nan], color='k', linestyle='-', linewidth=1)
ax2.plot([np.nan, np.nan], [np.nan, np.nan], color='k', linestyle='-', linewidth=1)

# Set x-axis limits
ax.set_xlim(np.min(binned_wvb), np.max(binned_wvb))
ax2.set_xlim(np.min(binned_wvr), np.max(binned_wvr))

# Set y-axis limits based on the range of normalized flux values
ax.set_ylim(min_flux, max_flux)
ax2.set_ylim(min_flux, max_flux)

# Hide the spines between ax and ax2
ax.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax.yaxis.tick_left()
ax2.tick_params(labelright='off')
ax2.yaxis.tick_right()

# Adjust the figure layout
f.text(0.5, 0.03, 'Wavelength [Angstrom]', ha='center', size=12)
f.text(0.065, 0.5, 'Normalized Flux', va='center', rotation='vertical', size=12)
f.suptitle('Normalized Spectra (Broken Axis)', size=14)

plt.show()


IndexError: boolean index did not match indexed array along dimension 1; dimension is 3854 but corresponding boolean dimension is 3853